In [1]:
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
from konlpy.tag import Okt
from tqdm import tqdm
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
import folium
from matplotlib import font_manager, rc

# 처음 사용자 입력 시 음식명으로 검색은 0, 재료명으로 검색은 1로 선택하게 하게 함.
# 0(음식명) 입력하면 음식 카테고리 리스트를, 1(재료명) 입력하면 재료 카테고리 리스트를  보여주고 다시 선택하게 함.
# 우선 0 -> 음식 카테고리 선택 -> 최종 음식명을 선택하게 됨
# 최종 음식명을 선택하게 되면 같은 이름의 요리명을 가진 레시피를 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 1을 선택하면 -> 재료 카테고리 선택 -> 최종 주 재료명을 선택하게 됨
# 최종 재료명을 선택하게 되면 해당 재료가 들어있는 요리를 찾아 조회수와 스크랩수가 높은 순으로 3개를 추천해주는 시스템.
# 이후 좀더 구체적으로 발전 할 계획.

In [ ]:
# 처음 사용자 입력 시 음식명으로 검색은 0, 재료명으로 검색은 1로 선택하게 함.

In [3]:
# 레시피 데이터 불러오기
reci = pd.read_excel('./240528_레시피.xlsx')
reci.head()


,레시피제목,요리명,요리재료내용,재료,조리순서,조회수,추천수,스크랩수,요리종류별명,요리재료별명
0,백전백승! 실패없는 김치볶음밥,김치볶음밥,[필수재료] 밥 1공기| 다진 양파 3큰술| 다진 김치 1/2컵| 다진 햄 3큰술 ...,"밥, 다진양파, 다진김치, 다진햄, 설탕, 김칫국물, 고추장, 부순참깨, 들기름",[조리순서]\n1.밥은 넓은 접시에 펼쳐 뜨거운 김을 날린다.밥 1공기\n2.양념장...,366065,1324,6914,밥/죽/떡,쌀
1,매일 먹는 밑반찬 메뉴로 추천!! 감자조림,감자조림,[필수재료] 감자 2개(360g)| 물 1cup [양념재료] 간장 5T| 설탕 1T...,"감자, 물, 간장, 설탕, 물엿, 다진마늘, 참깨, 참기름",[조리순서]\n1.감자는 손질하여 한입 크기로 썰어 준비한다.감자 2개(360g)감...,478527,1102,8221,밑반찬,채소류
2,새마을식당 7분김치찌개 만들기,김치찌개,[재료] 김치| 돼지고기(찌개용)| 양파 1/2개| 파 약간 [양념] 김치국물 2~...,"김치, 돼지고기, 양파, 파, 김치국물, 간장, 고춧가루, 다진마늘, 다시다",[조리순서]\n1.물 한 컵에 돼지고기를 넣고 끓여주세요물 대신 쌀뜨물을 사용하시면...,218649,1077,7594,찌개,채소류
3,백종원 제육볶음 레시피^^ 백종원의 노하우가 들어있는 손쉬운 레시피 백종원 제육볶음...,제육볶음,[재료] 돼지고기 600g| 양파 1개| 청양고추 2개| 대파 1뿌리 [양념] 설탕...,"돼지고기, 양파, 청양고추, 대파, 설탕, 고추장, 간장, 고춧가루, 다진마늘, 굴...",[조리순서]\n1.재료는 먹기 좋은 크기도 썰어서 준비해주세요^^\n2.고추장 2스...,976200,915,12418,메인반찬,돼지고기
4,비린내 걱정없는 백종원 고등어조림,고등어조림,[재료] 고등어 2마리| 무| 양파| 대파| 청양고추 [양념] 설탕 1큰술| 다진마...,"고등어, 무, 양파, 대파, 청양고추, 설탕, 다진마늘, 다진생강, 된장, 진간장,...",[조리순서]\n1.고등어는 내장을 제거하시고 핏물이 남지 않도록 깨끗이 씻어 준비합...,297242,887,10924,메인반찬,해물류


In [4]:
reci['요리종류별명'].unique()

array(['밥/죽/떡', '밑반찬', '찌개', '메인반찬', '면', '김치/젓갈/장류', '국/탕', '분식', '디저트',
       '피자'], dtype=object)

In [47]:
# 전체 함수
# 사용자의 입력을 받는 함수 정의
def get_user_choice():
    choice = input("음식명 검색은 0, 재료명 검색은 1을 입력해 주세요~^^")
    return choice


# 음식 카테고리 1 선택 함수 정의
def show_categories(df):
    categories = df['요리종류별명'].unique()
    for idx, category in enumerate(categories, start=1):
        print(f"{idx}.{category}")
    return categories


# 음식 카테고리 1 내 요리명 선택 함수 정의
def show_recipes_by_category(df, category):
    filterd_df = df[df['요리종류별명'] == category]
    recipes = filterd_df['요리명'].unique()
    for idx, recipe in enumerate(recipes, start=1):
        print(f"{idx}.{recipe}")
    return recipes


# 요리명 선택 후 추천 요리 방법 함수 정의
def recommend_recipes(df, recipe_name):
    filtered_df = df[df['요리명'].str.contains(recipe_name)]
    sorted_df = filtered_df.sort_values(by=['조회수', '스크랩수'], ascending=[False, False])
    top_recipes = sorted_df.head(3)
    
    responses = []
    for i, match in enumerate(top_recipes.iterrows(), start=1):  # 최대 3개까지 출력
        response = f"\n<{i}. {match[1]['레시피제목']}>\n\n재료:\n{match[1]['요리재료내용']}\n\n조리순서:\n{match[1]['조리순서']}"
        if i == 1:
            response = f"\n 쿠킹파트너 : {match[1]['요리명']} 레시피를 알려드리겠습니다.\n" + response
        responses.append(response)
    
    return '\n'.join(responses)

# 메인 실행 함수
def main():
    user_choice = get_user_choice()

    if user_choice == '0':
        categories = show_categories(reci)
        category_choice = input("1~10까지의 숫자를 입력하거나 카테고리명을 입력하세요: ")

        if category_choice.isdigit():
            category_choice = int(category_choice)
            if 1 <= category_choice <= len(categories):
                selected_category = categories[category_choice -1]
            else:
                print("다시 선택해주세요~")
                return
        else:
            if category_choice in categories:
                selected_category = category_choice
            else:
                print("다시 선택해주세요~")
                return
            
        recipes = show_recipes_by_category(reci, selected_category)
        recipe_choice = input( "숫자를 입력하거나 요리명을 입력하세요: ")
        
        if recipe_choice.isdigit():
            recipe_choice = int(recipe_choice)
            if 1 <= recipe_choice <= len(recipes):
                selected_recipe = recipes[recipe_choice -1]
            else:
                print("다시 선택해주세요~")
                return
            
        else:
            if recipe_choice in recipes:
                selected_recipe =recipe_choice
            else:
                print("다시 선택해주세요~")
                return
            
        
        print(recommend_recipes(reci, selected_recipe))


    elif user_choice == '1':
        print('재료명 검색은 아직.....')
    else:
        print("0과 1중에 입력해주세요")

if __name__ == "__main__":
    main()

재료명 검색은 아직.....
